In [ ]:
import duckdb
import pandas as pd

In [ ]:
def merge_idealista_tables(db_path):

    conn = duckdb.connect(db_path)
    
    # Query to get all table names containing 'idealista'
    query = "SELECT table_name FROM information_schema.tables WHERE table_name ILIKE '%idealista%';"
    table_names = conn.execute(query).fetchall()

    # Extract table names from query result
    table_names = [name[0] for name in table_names]

    # Read each table into a DataFrame and merge them
    merged_df = pd.DataFrame()
    for table_name in table_names:
        df = conn.execute(f'SELECT * FROM {table_name}').fetchdf()
        merged_df = pd.concat([merged_df, df], ignore_index=True)

    # Close the connection
    conn.close()
    
    return merged_df


In [ ]:
def save_merged_table_to_new_db(merged_df, new_db_path, table_name='idealista'):
    # Connect to the new DuckDB database
    conn = duckdb.connect(new_db_path)
    
    # Save the merged DataFrame to the new database
    conn.execute(f"CREATE TABLE {table_name} AS SELECT * FROM merged_df")
    
    # Close the connection
    conn.close()

In [ ]:
merged_dataframe = merge_idealista_tables('../formatted_zone/formatted.duckdb')
save_merged_table_to_new_db(merged_dataframe, '../trusted_zone/trusted.duckdb')